In [6]:
import torch
from transformers import GPT2Model, GPT2Tokenizer
from sklearn.decomposition import PCA
import numpy as np

# Load model and tokenizer, set padding token
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token (<|endoftext|>)
model = GPT2Model.from_pretrained('gpt2', output_hidden_states=True)  # Enable hidden states
prompt = "transform a grid: flip horizontal then rotate 90 degrees"
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=20)
outputs = model(**inputs)  # Get model outputs
embeddings = outputs.hidden_states[8]  # Layer 8, token-level embeddings

# Reshape for PCA: [seq_len, hidden_size] by removing batch dim
embeddings = embeddings.squeeze(0)  # Shape: [seq_len, hidden_size]
n_tokens = embeddings.shape[0]
print(f"Number of tokens: {n_tokens}")
if n_tokens < 3:
    raise ValueError(f"Number of tokens ({n_tokens}) is less than n_components=3. Consider increasing max_length.")

# Warp: PCA reduction and funnel fit
pca = PCA(n_components=3)
whitened = pca.fit_transform(embeddings.detach().numpy())
r = np.sqrt(whitened[:, 0]**2 + whitened[:, 1]**2)  # Radial distance
bins = np.linspace(r.min(), r.max(), n_tokens)
z_fit = np.interp(r, bins, -np.minimum.accumulate(np.percentile(whitened, q=np.linspace(0, 100, n_tokens), axis=0).mean(axis=1)))
z_tmpl = - (r**2) / (r.max()**2 + 1e-8) * abs(z_fit.min())
z_prof = 0.5 * z_fit + 0.5 * z_tmpl  # Blended profile
print(f"z_prof shape: {z_prof.shape}, min/max: {z_prof.min(), z_prof.max()}")

# Simulate traversal (cap phantom and adjust weights)
d = 3
T = 600
dt = 0.02
m = 4.0
lamb = 0.35
gamma = 0.04
a_damp = 1 - gamma * dt / (2 * m)

p = np.eye(d)  # Prototypes: flip h, flip v, rotate
c = np.array([[0.5, 0, 0], [0, 0.005, 0], [0, 0, 0.5]])  # Maintain hijack
w = np.zeros((T, 3))
w[100:250, 0] = np.linspace(0, 2.0, 150)  # Extended flip h
w[250:400, 2] = np.linspace(0, 2.0, 150)  # Shift rotate after flip h

# Normalize initial position
y = whitened[0].copy() / np.linalg.norm(whitened[0])  # Start normalized
v = np.zeros(d)
Y = np.zeros((T, d))
for t in range(T - 1):
    grad_U = np.zeros(d)
    for k in range(3):
        Pi = np.eye(d) - np.outer(p[:, k], p[:, k])
        grad_U += w[t, k] * Pi @ (y - c[:, k])
    v_half = a_damp * v - (lamb * dt / (2 * m)) * grad_U
    y_next = y + dt * v_half
    grad_U_next = np.zeros(d)
    for k in range(3):
        Pi = np.eye(d) - np.outer(p[:, k], p[:, k])
        grad_U_next += w[t + 1, k] * Pi @ (y_next - c[:, k])
    v_next = a_damp * v_half - (lamb * dt / (2 * m)) * grad_U_next
    y = y_next
    v = v_next
    Y[t] = y
print(f"Final y position: {y}")

# Detect: Energies and matched filtering with cap
E_par = np.zeros((T, 3))
E_perp = np.zeros((T, 3))
for t in range(T):
    for k in range(3):
        s = np.dot(p[:, k], Y[t] - c[:, k])
        E_par[t, k] = s**2
        E_perp[t, k] = min(np.linalg.norm(Y[t] - c[:, k])**2 - E_par[t, k], 1.0)  # Cap at 1.0
print(f"Max E_perp: {np.max(E_perp, axis=0)}")

# Light denoise: EMA smoothing
gamma_ema = 0.9
E_perp_smooth = np.zeros_like(E_perp)
E_perp_smooth[0] = E_perp[0]
for t in range(1, T):
    E_perp_smooth[t] = gamma_ema * E_perp_smooth[t-1] + (1 - gamma_ema) * E_perp[t]
print(f"Max E_perp_smooth: {np.max(E_perp_smooth, axis=0)}")

# Exclusive residuals and z-scoring
Z = (E_perp_smooth - E_perp_smooth.mean(axis=0)) / (E_perp_smooth.std(axis=0) + 1e-8)
E = np.zeros((T, 3))
for k in range(3):
    Q_minus = np.hstack([Z[:, j][:, np.newaxis] for j in range(3) if j != k]) if any(j != k for j in range(3)) else np.zeros((T, 0))
    if Q_minus.shape[1] > 0:
        Q, _ = np.linalg.qr(Q_minus)
        proj = Q @ (Q.T @ Z[:, k])
    else:
        proj = np.zeros(T)
    rex = np.maximum(Z[:, k] - proj, 0)
    E[:, k] = np.convolve(rex, np.ones(7)/7, mode='same')

# Matched filter
tau = np.arange(50)
q = np.sin(np.pi * tau / 50)
q = q - q.mean()
flipped_q = q[::-1]
C = np.zeros((T, 3))
for k in range(3):
    num = np.convolve(E[:, k] - E[:, k].mean(), flipped_q, mode='same')
    den1 = np.sqrt(np.convolve((E[:, k] - E[:, k].mean())**2, np.ones(len(q)), mode='same'))
    den2 = np.sqrt(len(q) * np.var(q))
    den = den1 * den2 + 1e-8
    C[:, k] = num / den

max_C = np.max(C, axis=0)
t_star = np.argmax(C, axis=0)
print(f"Max correlations: {max_C}")

# Null calibration (simplified z-score)
z_scores = np.zeros(3)
for k in range(3):
    null_maxs = [np.max(np.convolve(np.roll(E[:, k], shift) - E[:, k].mean(), flipped_q, mode='same')) for shift in range(1, 65)]
    z_scores[k] = (max_C[k] - np.mean(null_maxs)) / (np.std(null_maxs) + 1e-8)
print(f"Z-scores: {z_scores}")

# Area and thresholds
A = np.zeros(3)
for k in range(3):
    max_E = E[t_star[k], k]
    thresh = max_E / 2
    left = t_star[k]
    while left > 0 and E[left, k] > thresh and left > t_star[k] - 100:
        left -= 1
    right = t_star[k]
    while right < T - 1 and E[right, k] > thresh and right < t_star[k] + 100:
        right += 1
    A[k] = np.sum(E[left:right + 1, k])
print(f"Areas: {A}")

tau_area = 0.05
tau_corr = 0.25
tau_z = 5000.0  # Adjusted for high z-score artifact
detected = [k for k in range(3) if A[k] > tau_area and max_C[k] > tau_corr and z_scores[k] > tau_z]

print("Detected primitives (0: flip h, 1: flip v, 2: rotate):", detected)

Number of tokens: 10
z_prof shape: (10,), min/max: (0.0, 63.99386876821518)
Final y position: [-1.60235818e-01  1.98211648e-04 -4.00831505e-06]
Max E_perp: [3.93039242e-08 1.00000000e+00 1.00000000e+00]
Max E_perp_smooth: [3.6079832e-08 1.0000000e+00 1.0000000e+00]
Max correlations: [0.39093386 0.2068642  0.42910282]
Z-scores: [-1.49004491e+00  1.52112233e+07  1.48021288e+07]
Areas: [1.92994647e+01 9.21114413e-07 9.07736256e-07]
Detected primitives (0: flip h, 1: flip v, 2: rotate): []
